In [1]:
from Explanations_Models.Custom_DT.LIME import LIME
from FeatureImportance.FI import FeatureImportance
import yaml
import argparse
import os
import sys
import numpy as np
import torch
import gym
import matplotlib
import matplotlib.pyplot as plt
import unittest
from DeepLearning_Models.utils.general import join, plot_combined
from DeepLearning_Models.ActorCritic.policy_gradient import PolicyGradient
from EnvRunner import GymRunner
import json
import warnings
from tqdm import tqdm

# Suppress all deprecation warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=UserWarning)

yaml.add_constructor("!join", join)
parser = argparse.ArgumentParser()

parser.add_argument("--config_filename", required=False, type=str)

_StoreAction(option_strings=['--config_filename'], dest='config_filename', nargs=None, const=None, default=None, type=<class 'str'>, choices=None, help=None, metavar=None)

In [2]:

config_file = open("config_envs/{}.yml".format("_lunar_lander"))
config = yaml.load(config_file, Loader=yaml.FullLoader)
config.update(yaml.load(open("config_explanations/{}.yml".format("_lunar_lander"), encoding="utf8"), Loader= yaml.FullLoader))
Runner = GymRunner(config)
Runner.load_weights(PATH = None)

device:  cpu


c:\Users\blav1\Scripts\BryansClaimToFame\InterpretableModelExploration\EnvRunner.py:58: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.baseline_network.load_state_

In [3]:
config["surrogate"]["use_FI"] = True
config["sampler"]["num_samples"] = 20
config["FI"]["grouping"] = "Max_avg"

FI = FeatureImportance(config, Runner.model.network)
DTMode = LIME(config, Runner, FI_getta=FI)
#DTMod = LIME(config, Runner)
X,Y = DTMode.sample_set()


In [4]:
out, FI_out = FI.Relevence(X, h = .5)
FI_out

0.5


array([[[4.50962074e-02, 3.58811706e-01, 9.23777744e-02, 5.65175831e-01,
         9.25378799e-02, 2.99558435e-02, 1.52548887e-02, 1.50913805e-01],
        [3.73007953e-01, 2.09140763e-01, 7.15429962e-01, 4.01259422e-01,
         9.19306040e-01, 8.07571650e-01, 2.06775278e-01, 3.86880308e-01],
        [1.05154797e-01, 4.80010152e-01, 3.88781987e-02, 7.28065491e-01,
         1.89774837e-02, 5.27935363e-02, 4.17410657e-02, 1.74671441e-01],
        [3.13299924e-01, 2.78001755e-01, 4.03197616e-01, 3.77511859e-01,
         5.47171891e-01, 5.45387089e-01, 1.59822121e-01, 8.84426683e-02]],

       [[5.89077100e-02, 4.01527584e-01, 6.30915165e-02, 5.85912287e-01,
         6.48523048e-02, 3.66035220e-03, 1.18080257e-02, 1.48149997e-01],
        [4.19452548e-01, 2.04703689e-01, 7.17371643e-01, 3.30864757e-01,
         9.22625840e-01, 8.05771768e-01, 2.04830959e-01, 4.01640594e-01],
        [1.31617710e-01, 5.52318573e-01, 2.99787172e-03, 7.75525749e-01,
         1.89211946e-02, 9.87069607e-02, 4.

In [5]:
print(config["surrogate"]["multi_tree"])
config["surrogate"]["multi_tree"] = False

False


In [6]:
Metrics = {}
for i in ["Max_avg", "Max_all", "Var_weighted", "Baseline", "class"]:
    Metrics[i] = []
NumEpochs = 1
num_samples = [5,10,20]
for samps in num_samples:
    print(samps)
    for ep in range(NumEpochs):
        config["surrogate"]["use_FI"] = True
        config["sampler"]["num_samples"] = samps
        config["FI"]["grouping"] = "Max_avg"

        FI = FeatureImportance(config, Runner.model.network)
        DTMode = LIME(config, Runner, FI_getta=FI)
        #DTMod = LIME(config, Runner)
        X,Y = DTMode.sample_set()
        config["surrogate"]["use_FI"] = False
        config["sampler"]["num_samples"] = samps
        config["FI"]["grouping"] = "Baseline"
        DTMode = LIME(config, Runner)
        DTMode.surr_model.fit(X,Y)
        # DTMode.percent_Correct()
        # DTMode.uniform_Correct()
        print(config["FI"]["grouping"],DTMode.get_metrics())
        Metrics[config["FI"]["grouping"]].append(DTMode.get_metrics())
        
        config["surrogate"]["use_FI"] = True
        config["sampler"]["num_samples"] = samps
        config["FI"]["grouping"] = "Max_avg"
        FI = FeatureImportance(config, Runner.model.network)
        DTMode = LIME(config, Runner,FI_getta=FI)
        DTMode.surr_model.fit(X,Y)
        #DTMode.percent_Correct()
        #DTMode.uniform_Correct()
        print(config["FI"]["grouping"],DTMode.get_metrics())
        Metrics[config["FI"]["grouping"]].append(DTMode.get_metrics())

        config["surrogate"]["use_FI"] = True
        config["sampler"]["num_samples"] = samps
        config["FI"]["grouping"] = "Var_weighted"
        FI = FeatureImportance(config, Runner.model.network)
        DTMode = LIME(config, Runner,FI_getta=FI)
        DTMode.surr_model.fit(X,Y)
        #DTMode.percent_Correct()
        #DTMode.uniform_Correct()
        print(config["FI"]["grouping"],DTMode.get_metrics())
        Metrics[config["FI"]["grouping"]].append(DTMode.get_metrics())

        config["surrogate"]["use_FI"] = True
        config["sampler"]["num_samples"] = samps
        config["FI"]["grouping"] = "Max_all"
        FI = FeatureImportance(config, Runner.model.network)
        DTMode = LIME(config, Runner,FI_getta=FI)
        DTMode.surr_model.fit(X,Y)
        #DTMode.percent_Correct()
        #DTMode.uniform_Correct()
        print(config["FI"]["grouping"],DTMode.get_metrics())
        Metrics[config["FI"]["grouping"]].append(DTMode.get_metrics())

        config["surrogate"]["use_FI"] = True
        config["sampler"]["num_samples"] = samps
        config["FI"]["grouping"] = "class"
        FI = FeatureImportance(config, Runner.model.network)
        DTMode = LIME(config, Runner,FI_getta=FI)
        DTMode.surr_model.model.single_tree = True
        DTMode.surr_model.fit(X,Y)
        #DTMode.percent_Correct()
        #DTMode.uniform_Correct()
        print(config["FI"]["grouping"],DTMode.get_metrics())
        Metrics[config["FI"]["grouping"]].append(DTMode.get_metrics())

5
Baseline {'Policy_Captured': 0.1794682418691322, 'Uniform_Captured': 0.1909, 'Representation': 1.25, 'Depth': 4, 'Breadth': 2}
0.0001
Max_avg {'Policy_Captured': 0.1438166708674162, 'Uniform_Captured': 0.2536, 'Representation': 1.25, 'Depth': 3, 'Breadth': 4}
0.0001
Var_weighted {'Policy_Captured': 0.14639973191802744, 'Uniform_Captured': 0.2536, 'Representation': 1.25, 'Depth': 3, 'Breadth': 4}
0.0001
Max_all {'Policy_Captured': 0.1846200363992752, 'Uniform_Captured': 0.2536, 'Representation': 1.25, 'Depth': 3, 'Breadth': 4}
0.0001
class {'Policy_Captured': 0.17264583237048717, 'Uniform_Captured': 0.3683, 'Representation': 1.25, 'Depth': 3, 'Breadth': 4}
10
Baseline {'Policy_Captured': 0.2923258184031536, 'Uniform_Captured': 0.2658, 'Representation': 2.5, 'Depth': 4, 'Breadth': 2}
0.0001
Max_avg {'Policy_Captured': 0.31846715708467793, 'Uniform_Captured': 0.3745, 'Representation': 2.5, 'Depth': 4, 'Breadth': 2}
0.0001
Var_weighted {'Policy_Captured': 0.3288147814197771, 'Uniform_Cap

In [5]:
Metrics = {}
for i in ["Max_avg", "Max_all", "Var_weighted", "Baseline", "class"]:
    Metrics[i] = []
NumEpochs = 1
num_samples = [10]
for samps in num_samples:
    print(samps)
    for ep in range(NumEpochs):
        config["surrogate"]["use_FI"] = True
        config["sampler"]["num_samples"] = samps
        config["FI"]["grouping"] = "Max_avg"

        FI = FeatureImportance(config, Runner.model.network)
        DTMode = LIME(config, Runner, FI_getta=FI)
        #DTMod = LIME(config, Runner)
        X,Y = DTMode.sample_set()
        config["surrogate"]["use_FI"] = False
        config["sampler"]["num_samples"] = samps
        config["FI"]["grouping"] = "Baseline"
        DTMode = LIME(config, Runner)
        DTMode.surr_model.model.single_tree = True
        DTMode.surr_model.fit(X,Y)
        # DTMode.percent_Correct()
        # DTMode.uniform_Correct()
        print(config["FI"]["grouping"],DTMode.get_metrics())
        Metrics[config["FI"]["grouping"]].append(DTMode.get_metrics())

        config["surrogate"]["use_FI"] = True
        config["sampler"]["num_samples"] = samps
        config["FI"]["grouping"] = "Max_avg"
        DTMode = LIME(config, Runner,FI_getta=FI)
        DTMode.surr_model.model.single_tree = True
        DTMode.surr_model.fit(X,Y)
        #DTMode.percent_Correct()
        #DTMode.uniform_Correct()
        print(config["FI"]["grouping"],DTMode.get_metrics())
        Metrics[config["FI"]["grouping"]].append(DTMode.get_metrics())

        config["surrogate"]["use_FI"] = True
        config["sampler"]["num_samples"] = samps
        config["FI"]["grouping"] = "Var_weighted"
        DTMode = LIME(config, Runner,FI_getta=FI)
        DTMode.surr_model.model.single_tree = True
        DTMode.surr_model.fit(X,Y)
        #DTMode.percent_Correct()
        #DTMode.uniform_Correct()
        print(config["FI"]["grouping"],DTMode.get_metrics())
        Metrics[config["FI"]["grouping"]].append(DTMode.get_metrics())

        config["surrogate"]["use_FI"] = True
        config["sampler"]["num_samples"] = samps
        config["FI"]["grouping"] = "Max_all"
        DTMode = LIME(config, Runner,FI_getta=FI)
        DTMode.surr_model.model.single_tree = True
        DTMode.surr_model.fit(X,Y)
        #DTMode.percent_Correct()
        #DTMode.uniform_Correct()
        print(config["FI"]["grouping"],DTMode.get_metrics())
        Metrics[config["FI"]["grouping"]].append(DTMode.get_metrics())

        config["surrogate"]["use_FI"] = True
        config["sampler"]["num_samples"] = samps
        config["FI"]["grouping"] = "class"
        DTMode = LIME(config, Runner,FI_getta=FI)
        DTMode.surr_model.model.single_tree = True
        DTMode.surr_model.fit(X,Y)
        #DTMode.percent_Correct()
        #DTMode.uniform_Correct()
        print(config["FI"]["grouping"],DTMode.get_metrics())
        Metrics[config["FI"]["grouping"]].append(DTMode.get_metrics())

10
Baseline {'Policy_Captured': 0.6755, 'Uniform_Captured': 0.3905, 'Representation': 2.5, 'Depth': 4, 'Breadth': 2}


c:\Users\blav1\Scripts\BryansClaimToFame\InterpretableModelExploration\Explanations_Models\Custom_DT\Custom_DT_Pack\Node.py:149: RuntimeWarning: divide by zero encountered in divide
  val = (1/FI_val[var].iloc[0]) * val
c:\Users\blav1\Scripts\BryansClaimToFame\InterpretableModelExploration\Explanations_Models\Custom_DT\Custom_DT_Pack\Node.py:149: RuntimeWarning: invalid value encountered in scalar multiply
  val = (1/FI_val[var].iloc[0]) * val


Max_avg {'Policy_Captured': 0.6456666666666667, 'Uniform_Captured': 0.7059, 'Representation': 2.5, 'Depth': 4, 'Breadth': 2}


c:\Users\blav1\Scripts\BryansClaimToFame\InterpretableModelExploration\Explanations_Models\Custom_DT\Custom_DT_Pack\Node.py:149: RuntimeWarning: divide by zero encountered in divide
  val = (1/FI_val[var].iloc[0]) * val
c:\Users\blav1\Scripts\BryansClaimToFame\InterpretableModelExploration\Explanations_Models\Custom_DT\Custom_DT_Pack\Node.py:149: RuntimeWarning: invalid value encountered in scalar multiply
  val = (1/FI_val[var].iloc[0]) * val


Var_weighted {'Policy_Captured': 0.647, 'Uniform_Captured': 0.7059, 'Representation': 2.5, 'Depth': 4, 'Breadth': 2}


c:\Users\blav1\Scripts\BryansClaimToFame\InterpretableModelExploration\Explanations_Models\Custom_DT\Custom_DT_Pack\Node.py:149: RuntimeWarning: divide by zero encountered in divide
  val = (1/FI_val[var].iloc[0]) * val
c:\Users\blav1\Scripts\BryansClaimToFame\InterpretableModelExploration\Explanations_Models\Custom_DT\Custom_DT_Pack\Node.py:149: RuntimeWarning: invalid value encountered in scalar multiply
  val = (1/FI_val[var].iloc[0]) * val


Max_all {'Policy_Captured': 0.6528333333333335, 'Uniform_Captured': 0.7059, 'Representation': 2.5, 'Depth': 4, 'Breadth': 2}


c:\Users\blav1\Scripts\BryansClaimToFame\InterpretableModelExploration\Explanations_Models\Custom_DT\Custom_DT_Pack\Node.py:149: RuntimeWarning: divide by zero encountered in divide
  val = (1/FI_val[var].iloc[0]) * val
c:\Users\blav1\Scripts\BryansClaimToFame\InterpretableModelExploration\Explanations_Models\Custom_DT\Custom_DT_Pack\Node.py:149: RuntimeWarning: invalid value encountered in scalar multiply
  val = (1/FI_val[var].iloc[0]) * val


class {'Policy_Captured': 0.6485, 'Uniform_Captured': 0.7059, 'Representation': 2.5, 'Depth': 4, 'Breadth': 2}


In [12]:
config["surrogate"]["use_FI"] = True
config["sampler"]["num_samples"] = 20
config["FI"]["grouping"] = "Max_avg"
DTMode = LIME(config, Runner,FI_getta=FI)
DTMode.surr_model.model.single_tree = True
DTMode.surr_model.fit(X,Y)
#DTMode.percent_Correct()
#DTMode.uniform_Correct()
print(DTMode.get_metrics())

{'Policy_Captured': 0.905, 'Uniform_Captured': 0.8111, 'Representation': 5.0, 'Depth': 4, 'Breadth': 2}


In [ ]:
#np.linalg.norm(X)
NumNorm = 3
norms = np.linalg.norm(X - X[3,:],axis=1)
print(norms)
closest_points = np.argsort(norms)
print(closest_points)
print(X[closest_points[1:NumNorm+1]])
X[3]


In [12]:
import pandas as pd
def fit_multi_tree(X,Y):
    X = pd.DataFrame(X, columns=config["picture"]["labels"])
    Y_set =[]
    for i in range(Y.shape[1]):
        Y_ind = Y[:,i]
        
        Y_ind = pd.DataFrame(Y_ind, columns=[config["picture"]["class_names"][i]])
        Y_set.append(Y_ind)
    for df in Y_set:
        

     leg1hip
0  -1.184277
1   1.074105
2   0.067084
3  -0.566440
4  -0.411135
5   2.650956
6   2.516359
7   1.593475
8   0.600363
9  -1.657920
10  1.001916
11 -0.120257
12 -0.671482
13  1.443364
14  2.785102
15 -2.105793
16 -2.189750
17 -1.235336
18  1.244805
19  1.019308
    leg1knee
0   4.281791
1   1.049716
2   4.373060
3   4.644642
4   3.873209
5  -0.985386
6   0.414858
7   0.504745
8   0.614699
9   5.468406
10  1.301445
11 -0.765672
12  4.251453
13  1.037093
14 -1.448827
15  5.371675
16  4.024936
17  3.412257
18 -1.186071
19  0.428277
     leg2hip
0   1.117309
1   3.041218
2   0.456903
3   0.470654
4   0.392121
5   0.118636
6   0.012060
7   1.071565
8   2.719003
9   1.161066
10  1.637380
11  1.026187
12  0.437998
13  0.713088
14  0.318753
15  1.931831
16  1.195518
17  0.369234
18  1.617476
19  1.115166
    leg2knee
0   1.479223
1   0.562052
2  -0.163588
3  -0.379814
4  -0.476035
5  -1.503159
6  -1.534957
7   0.571014
8   1.573693
9   1.091411
10  0.381035
11  0.733117
12  0.072092

In [ ]:
def fit_multi_tree(self, X,Y):
        X = pd.DataFrame(X, columns=self.config["picture"]["labels"])
        Y_set =[]
        for i in range(Y.shape[1]):
            Y_ind = Y[:,i]
            Y_ind = pd.DataFrame(Y_ind, columns=self.config["picture"]["class_names"][i])
            Y_set.append(Y_ind)
        print(Y_set)

In [31]:
#np.linalg.norm(X)
NumNorm = 3
norms = np.linalg.norm(X - X[3,:],axis=1)
print(norms)
closest_points = np.argsort(norms)
print(closest_points)
print(X[closest_points[1:NumNorm+1]])
X[3]

# smallest_x = np.sort(np.linalg.norm(X - X[0,:], axis=1))


[3.5472643  2.6999311  2.0795522  0.         0.47523844 3.1976788
 2.294066   1.2353044  2.9847898  3.3937418  0.3390295  0.5805512
 1.5873252  0.2678124  2.7930048  1.5496327  1.426849   1.0590192
 2.1165824  3.4515421 ]
[ 3 13 10  4 11 17  7 16 15 12  2 18  6  1 14  8  5  9 19  0]
[[ 8.7027878e-02  2.6118262e-02  5.7043105e-01  2.4200780e-03
   2.1383483e-02  9.9994922e-01 -5.4509425e-01 -9.9999672e-01
   0.0000000e+00  1.1351696e+00  1.1324883e-06 -1.2880957e-01
  -6.6493994e-01  1.0000000e+00  3.3688778e-01  3.4025449e-01
   3.4990591e-01  3.6822218e-01  3.9869690e-01  4.5358717e-01
   5.3456855e-01  6.6646701e-01  9.2623001e-01  1.0000000e+00]
 [-2.8283965e-02  3.6027487e-02  5.4276425e-01  5.2066308e-02
   4.4382095e-02  1.0000005e+00 -4.9516773e-01 -1.0000001e+00
   0.0000000e+00  1.1261002e+00 -1.4901161e-08  7.7043772e-03
  -1.0000002e+00  1.0000000e+00  3.5446528e-01  3.6189559e-01
   3.7697279e-01  3.9752460e-01  4.3017569e-01  4.8260763e-01
   5.6276029e-01  6.8090039e-01  

array([ 5.9509367e-02,  2.5702687e-02,  5.3696597e-01, -5.4441798e-03,
        1.8686780e-01,  9.9884343e-01, -6.7270041e-01, -9.9980217e-01,
        0.0000000e+00,  1.1359957e+00,  2.6330352e-05, -9.8160863e-02,
       -8.2328790e-01,  1.0000000e+00,  3.3734119e-01,  3.3890429e-01,
        3.5152331e-01,  3.7168124e-01,  4.0349156e-01,  4.5370671e-01,
        5.3601521e-01,  6.6819835e-01,  9.1875863e-01,  1.0000000e+00],
      dtype=float32)

In [3]:
# config["surrogate"]["use_FI"] = True
# config["sampler"]["num_samples"] = 60
# config["FI"]["grouping"] = "Var_weighted"
# mets = []
# for i in range(0,25):
#     FI = FeatureImportance(config, Runner.model.network)
#     DTMode = LIME(config, Runner, FI_getta=FI)
#     #DTMod = LIME(config, Runner)
#     DTMode.train()
#     mets.append(DTMode.get_metrics())
# # DTMode.percent_Correct(print_val=True)
# # D = DTMode.surr_model.model.get_dict_representation()
# # print(D)
# # for i in range(0, 10):
# #     DTMode.train()
# #     thang.append(DTMode.percent_Correct())
# #     print(np.mean(thang), np.var(thang))


In [4]:

config["surrogate"]["use_FI"] = True
config["sampler"]["num_samples"] = 20
config["FI"]["grouping"] = "Max_avg"

FI = FeatureImportance(config, Runner.model.network)
DTMode = LIME(config, Runner, FI_getta=FI)
#DTMod = LIME(config, Runner)
X,Y = DTMode.sample_set()
DTMode.surr_model.fit(X,Y)
print(config["FI"]["grouping"], ": ", DTMode.get_metrics())

config["FI"]["grouping"] = "Var_weighted"
DTMode.surr_model.fit(X,Y)
print(config["FI"]["grouping"], ": ", DTMode.get_metrics())

config["FI"]["grouping"] = "Max_all"
DTMode.surr_model.fit(X,Y)
print(config["FI"]["grouping"], ": ", DTMode.get_metrics())

config["surrogate"]["use_FI"] = False
config["sampler"]["num_samples"] = 20

DTMode = LIME(config, Runner)
DTMode.surr_model.fit(X,Y)
print("NOT using FI: ",DTMode.get_metrics())


Max_avg :  {'Policy_Captured': 4.7586703, 'Uniform_Captured': 129.4437, 'Representation': 1.0, 'Depth': 11, 'Breadth': 4}
Var_weighted :  {'Policy_Captured': 5.1761966, 'Uniform_Captured': 129.36089, 'Representation': 1.0, 'Depth': 11, 'Breadth': 4}
Max_all :  {'Policy_Captured': 4.3388534, 'Uniform_Captured': 129.36089, 'Representation': 1.0, 'Depth': 11, 'Breadth': 4}
NOT using FI:  {'Policy_Captured': 1.7265011, 'Uniform_Captured': 107.68863, 'Representation': 1.0, 'Depth': 7, 'Breadth': 9}


In [ ]:
print(mets)

In [4]:
print(i)

23


In [13]:
config["surrogate"]["use_FI"] = False
config["sampler"]["num_samples"] = 60
mets = []
for i in range(0,25):
    FI = FeatureImportance(config, Runner.model.network)
    DTMode = LIME(config, Runner)
    #DTMod = LIME(config, Runner)
    DTMode.train()
    mets.append(DTMode.get_metrics())
# DTMode.percent_Correct(print_val=True)
# D = DTMode.surr_model.model.get_dict_representation()
# print(D)
# for i in range(0, 10):
#     DTMode.train()
#     thang.append(DTMode.percent_Correct())
#     print(np.mean(thang), np.var(thang))


{'Policy_Captured': 0.725, 'Uniform_Captured': 0.412, 'Representation': 7.5, 'Depth': 5, 'Breadth': 5}


In [ ]:
def visit_node(node):
    

ClassForgetting

In [9]:
#DTMod = LIME(config, Runner)
thang = []
for i in range(0, 10):
    DTMode.train()
    thang.append(DTMode.percent_Correct())
    print(np.mean(thang), np.var(thang))

0.9 0.0
0.7250000000000001 0.030624999999999996
0.7800000000000001 0.026466666666666663
0.7625000000000001 0.02076875
0.7090000000000001 0.028064
0.7291666666666666 0.025420138888888888
0.745 0.02329285714285714
0.764375 0.023008984375
0.78 0.022405555555555554
0.7925000000000001 0.02157125


what we got

In [10]:
thang1 = np.array(thang)
print(thang1.mean(), thang1.var())

0.7925000000000001 0.02157125


what we beat

In [11]:
thang2 = []
config["surrogate"]["use_FI"] = False
DTMod = LIME(config, Runner)

for i in range(0, 10):
    DTMod.train()
    thang2.append(DTMod.percent_Correct())

In [12]:
thang2 = np.array(thang2)
print(thang2.mean(), thang2.var())

0.7064999999999999 0.01735025


In [7]:
from scipy.stats import ttest_ind
t_statistic, p_value = ttest_ind(thang1, thang2)
print("T-statistic:", t_statistic)
print("P-value:", p_value)

NameError: name 'thang2' is not defined

In [8]:

out, matrix = FI.Calc_Feature_Importance(torch.tensor([1.,0.,1.,0.,1.,0.,1.,0.,1.,0.,1.,0.,1.,0.,1.,0.,1.,0.,1.,0.,1.,0.,1.,0.,]), printer=True)

point tensor([1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
        1., 0., 1., 0., 1., 0.])
output tensor([ 1.5826, -7.8118,  2.3578, -2.1897], grad_fn=<ViewBackward0>)
relevence [tensor([-0.4360,  0.0032, -0.2563,  0.0032, -0.5618,  0.0032,  0.5331,  0.0032,
        -0.1117,  0.0032,  0.4333,  0.0032,  0.4067,  0.0032,  0.5228,  0.0032,
         0.7383,  0.0032,  0.0690,  0.0032,  0.5256,  0.0032, -0.3183,  0.0032],
       grad_fn=<SqueezeBackward4>), tensor([ 4.6117, -0.0781,  0.9442, -0.0781,  5.6282, -0.0781, -3.1760, -0.0781,
         1.1894, -0.0781, -3.2217, -0.0781, -2.6642, -0.0781, -2.1719, -0.0781,
        -5.1672, -0.0781, -1.4828, -0.0781, -4.0037, -0.0781,  2.6397, -0.0781],
       grad_fn=<SqueezeBackward4>), tensor([-0.3696,  0.0232, -0.4396,  0.0232, -0.2606,  0.0232,  0.5377,  0.0232,
        -0.0442,  0.0232,  0.3225,  0.0232,  0.5381,  0.0232,  0.5866,  0.0232,
         0.5562,  0.0232,  0.2230,  0.0232,  0.6054,  0.0232, -0.1759,  0.0232]

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 3 dimensions. The detected shape was (2, 10, 4) + inhomogeneous part.

In [1]:
print(FI.shape)

NameError: name 'FI' is not defined

In [1]:
import pandas as pd

columns = pd.MultiIndex.from_product([config["picture"]["class_names"], config["picture"]["labels"]], names=['OutLogit', 'InLogit'])

# Reshape the data to (60, 8) to match the multi-level column structure
reshaped_data = FI.reshape(FI.shape[0], -1)

# Create the DataFrame
FI = pd.DataFrame(reshaped_data, columns=columns)
out = pd.DataFrame(out, columns=config["picture"]["class_names"])

NameError: name 'matrix' is not defined

In [3]:
DT = LIME(config, Runner)

In [ ]:
def print_activation(layer, input, output):
    print(f"Layer: {layer}, Activation Output: {output}")

# Register hooks to each layer
for layer in model.children():
    layer.register_forward_hook(print_activation)

# Dummy input to pass through the model
dummy_input = torch.randn(1, 10)  # Adjust size to match input layer
model(dummy_input)

In [5]:
Runner.model.network.forward(torch.tensor([[4.,4.,4.,4.,]]))


tensor([[-53.5020,  57.9378]], grad_fn=<AddmmBackward0>)

In [14]:
X,Y = DT.train(returner=True)


60


In [15]:
DT.percent_Correct(print_val=True)


1.1668402

In [16]:
DT.uniform_Correct(print_val=True)

111.059525

In [9]:
print(DT.surr_model.model.max_depth)
DT.surr_model.model.get_dict_representation()

10


{'Feature': 'Hip Joint 1 Speed',
 'Bucket': 0.5963983535766602,
 'Left_Child': {'Feature': 'Knee Joint 1 Angle',
  'Bucket': -0.16023868322372437,
  'Left_Child': {'Feature': 'Knee Joint 2 Angle',
   'Bucket': -0.030735313892364502,
   'Left_Child': {'Feature': 'Velocity X',
    'Bucket': 0.5160324573516846,
    'Left_Child': {'Feature': 'Hull Angle',
     'Bucket': -0.06656686216592789,
     'Left_Child': {'Value': array([-2.6837106 , -0.3007074 ,  1.4836516 ,  0.03225509], dtype=float32)},
     'Right_Child': {'Feature': 'Hull Angle',
      'Bucket': 0.0036161281168460846,
      'Left_Child': {'Value': array([-2.7936342 ,  2.177329  ,  1.6616061 ,  0.34213853], dtype=float32)},
      'Right_Child': {'Value': array([-1.1121528,  2.3536325,  2.649001 ,  1.4963849], dtype=float32)}}},
    'Right_Child': {'Feature': 'Hull Angle',
     'Bucket': 0.08883405476808548,
     'Left_Child': {'Feature': 'Hull Angle',
      'Bucket': 0.06528535485267639,
      'Left_Child': {'Value': array([-3.00

In [9]:
np.mean(Y, axis=0)

array([ 0.74338526,  1.4490379 ,  1.0475755 , -0.23259966], dtype=float32)

In [ ]:
from scipy.spatial.distance import pdist
import pandas as pd
Y = pd.DataFrame(Y, columns=config["picture"]["class_names"])
np.mean(pdist(Y.values, metric="euclidean"))

In [9]:
import pandas as pd
Y = np.array([[1., 4.2, 3.3]])
Y = pd.DataFrame(Y, columns = ["hit1", "hit2", "hit3"])

In [ ]:
from scipy.spatial.distance import pdist

np.mean(pdist(Y.values, metric="euclidean"))

In [31]:
a = torch.tensor([1., 2., 3.])
W = torch.tensor([[2., 3.,],[2., 3.,],[2., 3.,]])
b = torch.tensor([.5,.6])

In [32]:
print(a.shape, W.shape, b.shape)

torch.Size([3]) torch.Size([3, 2]) torch.Size([2])


In [33]:
a*W

RuntimeError: The size of tensor a (3) must match the size of tensor b (2) at non-singleton dimension 1

In [34]:
a@W+b

tensor([12.5000, 18.6000])

In [35]:
norm = []
CorVec = []
for i in range(len(b)):
    tmp = a*W[:,i] + b[i]
    norm.append(tmp.sum())
    CorVec.append(tmp)
    print(a*W[:,i] + b[i])
norm = torch.stack(norm)
CorVec = torch.stack(CorVec)
print(norm, CorVec)

tensor([2.5000, 4.5000, 6.5000])
tensor([3.6000, 6.6000, 9.6000])
tensor([13.5000, 19.8000]) tensor([[2.5000, 4.5000, 6.5000],
        [3.6000, 6.6000, 9.6000]])


In [36]:
norm += 1e-9

In [37]:
obj = torch.relu(a@W+b)

In [38]:
obj

tensor([12.5000, 18.6000])

In [39]:
normalized_CorVec = CorVec / norm.unsqueeze(1)

In [40]:
normalized_CorVec

tensor([[0.1852, 0.3333, 0.4815],
        [0.1818, 0.3333, 0.4848]])

In [41]:
out = obj @ normalized_CorVec
print(out)

tensor([ 5.6966, 10.3667, 15.0367])
